# Testing file 
### where we evaluate LAFTR's models using the test set

## Preliminaries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.data import Dataset
from tensorflow.keras.optimizers import Adam


from util.load_data import load_data
from util.evaluation import *
from models.madras_laftr.models import *
from models.madras_laftr.learning import train_loop as laftr_train

In [2]:
batch_size = 64
epochs = 100
learning_rate = 0.001

In [3]:
CLAS_COEFF = 1.
FAIR_COEFFS = [.2, .5, .7, 1.]
RECON_COEFF = 0.
hidden_layer_specs = {'clas':[8] , 'enc':[8] , 'dec':[8] , 'adv':[8]}

In [4]:
cv_seeds = [13, 29, 42, 55, 73]

## Load data

In [5]:
data_name = 'german'

In [6]:
x, y, a = load_data(data_name)
raw_data = (x, y, a)

In [7]:
xdim = x.shape[1]
ydim = y.shape[1]
adim = a.shape[1]
zdim = 8

## Result file

In [8]:
header = "model_name", "cv_seed", "fair_coeff", "clas_acc", "dp", "deqodds", "deqopp", "trade_dp", "trade_deqodds", "trade_deqopp", "TN_a0", "FP_a0", "FN_a0", "TP_a0", "TN_a1", "FP_a1", "FN_a1", "TP_a1"
results = []

## Testing loop
#### Each model is evalueted 5 times
#### In the end of each iteration we save the result

### LAFTR for DP

In [9]:
for cv_seed in cv_seeds:
    x_train, x_test, y_train, y_test, a_train, a_test = train_test_split(
        x, y, a, test_size=0.3, random_state=cv_seed)

    train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
    train_data = train_data.batch(batch_size, drop_remainder=True)

    test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
    test_data = test_data.batch(batch_size, drop_remainder=True)
    
    for FAIR_COEFF in FAIR_COEFFS:

        opt = Adam(learning_rate=learning_rate)

        model = DemParGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, test_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4DP', cv_seed, FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

        del(opt)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.13009534776210785 | 0.5940612554550171 | 1.018876075744629 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 2 | 0.13338442146778107 | 0.5937267541885376 | 0.9678672552108765 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 3 | 0.1375865787267685 | 0.5939900875091553 | 0.9061518311500549 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 4 | 0.14171598851680756 | 0.594352126121521 | 0.8460206985473633 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 5 | 0.1445290893316269 | 0.594110369682312 | 0.8026155829429626 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 6 | 0.14507092535495758 | 0.592994213104248 | 0.7889069318771362 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 7 | 0.1430077999830246 | 0.5909267663955688 | 0.8095170259475708 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 8 | 0.1398526281118393 | 0.5877128839492798 | 0.8407750129699707 | 0.774375081062316

### LAFTR for Eq Odds

In [10]:
for cv_seed in cv_seeds:
    x_train, x_test, y_train, y_test, a_train, a_test = train_test_split(
        x, y, a, test_size=0.3, random_state=cv_seed)

    train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
    train_data = train_data.batch(batch_size, drop_remainder=True)

    test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
    test_data = test_data.batch(batch_size, drop_remainder=True)

    for FAIR_COEFF in FAIR_COEFFS:

        opt = Adam(learning_rate=learning_rate)

        model = EqOddsUnweightedGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, test_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4EqOdds', cv_seed, FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

        del(opt)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | -0.14155283570289612 | 0.5940613746643066 | 1.0187199115753174 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 2 | -0.12446484714746475 | 0.593723714351654 | 0.9671182632446289 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 3 | -0.10371541976928711 | 0.593966543674469 | 0.9051128625869751 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 4 | -0.08409328013658524 | 0.5942866802215576 | 0.8465664982795715 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 5 | -0.07127205282449722 | 0.5940169095993042 | 0.8078330159187317 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 6 | -0.06913840025663376 | 0.5929819941520691 | 0.8003972768783569 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 7 | -0.0746518149971962 | 0.59136962890625 | 0.8153250217437744 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 8 | -0.0729827880859375 | 0.5895828604698181 | 0.8085312843322754 | 0.77437

### LAFTR for Eq Opp

In [11]:
for cv_seed in cv_seeds:
    x_train, x_test, y_train, y_test, a_train, a_test = train_test_split(
        x, y, a, test_size=0.3, random_state=cv_seed)

    train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
    train_data = train_data.batch(batch_size, drop_remainder=True)

    test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
    test_data = test_data.batch(batch_size, drop_remainder=True)

    for FAIR_COEFF in FAIR_COEFFS:

        opt = Adam(learning_rate=learning_rate)

        model = EqOppUnweightedGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, test_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4EqOpp', cv_seed, FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

        del(opt)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | -0.3052818477153778 | 0.5940489768981934 | 1.0187889337539673 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 2 | -0.26916784048080444 | 0.5937256813049316 | 0.9682203531265259 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 3 | -0.22606077790260315 | 0.5939565896987915 | 0.9084758758544922 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 4 | -0.18621458113193512 | 0.594115138053894 | 0.8531961441040039 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 5 | -0.1615041047334671 | 0.593616247177124 | 0.8183176517486572 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 6 | -0.16018055379390717 | 0.5924654006958008 | 0.8153252601623535 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 7 | -0.175368994474411 | 0.5908876657485962 | 0.8348792791366577 | 0.7743750810623169 | 0.6734375 | 0.2984375 | 0.0
> 8 | -0.17431455850601196 | 0.589178204536438 | 0.831702709197998 | 0.7743750

## Saving into DF then CSV

In [12]:
result_df = pd.DataFrame(results, columns=header)
result_df

,model_name,cv_seed,fair_coeff,clas_acc,dp,deqodds,deqopp,trade_dp,trade_deqodds,trade_deqopp,TN_a0,FP_a0,FN_a0,TP_a0,TN_a1,FP_a1,FN_a1,TP_a1
0,LAFTR4DP,13,0.2,0.722656,0.857973,0.824195,0.945093,0.784523,0.770093,0.819041,17.0,9.0,13.0,45.0,15.0,27.0,22.0,108.0
1,LAFTR4DP,13,0.5,0.714844,0.910576,0.875060,0.999204,0.800925,0.786879,0.833436,17.0,9.0,12.0,46.0,17.0,25.0,27.0,103.0
2,LAFTR4DP,13,0.7,0.718750,0.916390,0.886965,0.999204,0.805626,0.794046,0.836085,17.0,9.0,12.0,46.0,18.0,24.0,27.0,103.0
3,LAFTR4DP,13,1.0,0.722656,0.898394,0.890483,0.958621,0.800999,0.797839,0.824080,17.0,9.0,14.0,44.0,20.0,22.0,26.0,104.0
4,LAFTR4DP,29,0.2,0.691406,0.840378,0.873917,0.878553,0.758648,0.772022,0.773825,16.0,20.0,10.0,33.0,16.0,35.0,14.0,112.0
5,LAFTR4DP,29,0.5,0.691406,0.884002,0.915030,0.933001,0.775932,0.787653,0.794238,15.0,21.0,8.0,35.0,16.0,35.0,15.0,111.0
6,LAFTR4DP,29,0.7,0.687500,0.889652,0.918998,0.940938,0.775620,0.786570,0.794497,15.0,21.0,8.0,35.0,16.0,35.0,16.0,110.0
7,LAFTR4DP,29,1.0,0.691406,0.902310,0.938723,0.933001,0.782903,0.796304,0.794238,14.0,22.0,8.0,35.0,17.0,34.0,15.0,111.0
8,LAFTR4DP,42,0.2,0.675781,0.900059,0.878147,0.934615,0.771960,0.763787,0.784398,9.0,13.0,11.0,41.0,12.0,40.0,19.0,111.0
9,LAFTR4DP,42,0.5,0.679688,0.921592,0.906643,0.946154,0.782368,0.776930,0.791084,8.0,14.0,10.0,42.0,12.0,40.0,18.0,112.0


In [13]:
result_df.to_csv(f'{data_name}-result/laftr-{epochs}.csv')